## Finalize the output files

Edit variable names, metadata, etc.

### Imports

In [1]:
# --- Import Modules --- #

# Import Python Core Modules
import sys
import os
import time
import datetime

# Import Additional Modules
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd

tic = time.time()
print('Process initiated at {0}'.format(time.ctime()))
# --- End Import Modules --- #

Process initiated at Mon Apr  7 10:44:51 2025


In [2]:
in_nc = r'/path/to/outputs/agg_out/CONUS_HUC12_WB_combined_WY2011_2013.nc'

# Output directory
outDir = r'/path/to/outputs/agg_out/'

# Output files
out_nc = os.path.join(outDir, 'huc12_monthly_wb_iwaa_wrfhydro_WY2011_2013.nc')
#out_csv = os.path.join(outDir, 'huc12_monthly_wb_iwaa_wrfhydro_WY2011_2013_2.csv')

# Select output formats
write_NC = True      # Output netCDF file
#write_CSV = True     # Output CSV file

# Name the zone dimension
zone_name = 'WBDHU12'

# Name the time dimension
time_coord = 'time'

### Dictionaries to rename variables and set attributes

In [11]:
# Dictionary to rename variables. Also used to subset dataset by variable name.
var_rename_dict = {'totPRECIP':'Precip',
                   'totPRECIP':'PrecipLand',
                   'deltaACSNOW':'Snowfall',
                   'totqSfcLatRunoff':'Surfaceflow',
                   'totqBucket':'Baseflow',
                   'deltaACCET':'ET',
                   'avgSNEQV':'SWE',
                   'avgSOILM':'SoilWater',
                   'avgSOILSAT':'SoilSat',
                   'deltaUGDRNOFF':'Recharge',
                   'bucket_depth':'GWStore',
                   'Area_sqkm':'CatchmentArea',
                   'Precip':'Precip',
                   'landmask':'LandFraction',
                   'total_gridded_area': 'total_gridded_area',
                   'avgSOILM_wltadj_depthmean': 'avgSOILM_wltadj_depthmean',
                   'avgSOILSAT_wltadj_top1': 'avgSOILSAT_wltadj_top1',}

# Rename dimensions
rename_dim_dict = {zone_name:'huc_id'}

# Variable attributes dictionary
var_atts_dict = {'Precip':{'units':'mm',
                           'long_name':"Total monthly precipitation (land & water)"},
                'PrecipLand':{'units':'mm',
                              'long_name':"Total monthly precipitation (land only)"},
                'Snowfall':{'units':'mm',
                            'long_name':"Total monthly snowfall (land only)"},
                'Surfaceflow':{'units':'mm',
                               'long_name':"Total monthly surface flow"},
                'Baseflow':{'units':'mm',
                            'long_name':"Total monthly baseflow"},
                'ET':{'units':'mm',
                      'long_name':"Total monthly evapotranspiration (land only)"},
                'SWE':{'units':'mm',
                       'long_name':"Average monthly snow water equivalent (land only)"},
                'SoilWater':{'units':'mm',
                             'long_name':"Average monthly soil moisture in 2m soil column (land only)"},
                'SoilSat':{'units':'fraction',
                           'long_name':"Average monthly fractional soil saturation in 2m soil column (land only)"},
                'Recharge':{'units':'mm',
                            'long_name':"Total monthly recharge (land only)"},
                'GWStore':{'units':'mm',
                           'long_name':"Average monthly groundwater store"},
                'LandFraction':{'units':'fraction',
                                'standard_name':'area_fraction',
                                'long_name':"Land fraction of HUC12 from gridded data"},
                'CatchmentArea':{'units':'square kilometers',
                                 'long_name':"Total NWM catchment area (square kilometers)"},
                }

# Global attributes dictionary
out_global_atts = {'title':"HUC12 monthly WRF-Hydro modeling application",
                   'institution':"USGS",
                   'history':"Author, {}".format(time.ctime())
                  }   

In [4]:
ds = xr.open_dataset(in_nc)
ds

<xarray.Dataset> Size: 403MB
Dimensions:                    (WBDHU12: 86617, time: 35)
Coordinates:
  * WBDHU12                    (WBDHU12) int64 693kB 10100020101 ... 18100204...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/24)
    total_gridded_area         (WBDHU12) float64 693kB ...
    Precip                     (WBDHU12, time) float32 12MB ...
    landmask                   (WBDHU12) float64 693kB ...
    deltaACCET                 (WBDHU12, time) float64 24MB ...
    deltaACSNOW                (WBDHU12, time) float64 24MB ...
    deltaSNEQV                 (WBDHU12, time) float32 12MB ...
    ...                         ...
    Area_sqkm                  (WBDHU12) float64 693kB ...
    totInflow                  (time, WBDHU12) float64 24MB ...
    bucket_depth               (time, WBDHU12) float64 24MB ...
    totqBucket                 (time, WBDHU12) float64 24MB ...
    totqSfcLatRunoff           (time, WBDHU12) float64 24MB ...
    totStreamflow              (time, WBDHU12) float64 24MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Code to add back in Char HUCIDs from source file

In [5]:
%%time

# Convert the HUC dataset (polygons) to WGS84 to match the points
HUC_gpkg = r'/caldera/hovenweep/projects/usgs/water/impd/hytest/niwaa_wrfhydro_monthly_huc12_aggregations_sample_data/HUC12_grids/HUC12.gpkg'
HUC_gdf = gpd.read_file(HUC_gpkg, layer='WBDHU12_CONUS', ignore_geometry=True)

CPU times: user 2.37 s, sys: 906 ms, total: 3.28 s
Wall time: 3.31 s


### Assign the coordinates to match the string-type HUC12 IDs from the input feature class

In [6]:
%%time

# Create a new dataframe with just the HUC12 data
HUC_df = HUC_gdf[['HUC12', 'AREASQKM', 'STATES', 'TOHUC']].copy()

# Create new field that will match to the datatype in the input file
HUC_df['HUC12_int'] = HUC_df['HUC12'].astype(np.int64)

# Export the netCDF coordinate to a dataframe
nc_df = ds['WBDHU12'].to_dataframe()
nc_df.index = np.arange(nc_df.shape[0]) #reset_index()
combined_df = pd.merge(nc_df, HUC_df,  how='inner', left_on=['WBDHU12'], right_on=['HUC12_int'])

# Deal with duplicates
combined_df = combined_df[~combined_df.duplicated(subset=['HUC12_int'], keep='last')]

# Make sure they have the same number of values
assert combined_df['HUC12'].unique().shape == nc_df['WBDHU12'].unique().shape

# Make sure they are identical
assert (combined_df['HUC12_int'] == ds['WBDHU12'].data).sum() == ds['WBDHU12'].data.shape

da = xr.DataArray(combined_df['HUC12'].astype('S12'), coords={'WBDHU12': combined_df['HUC12'].astype('S12')},dims=['WBDHU12'])
ds['WBDHU12'] = da
#del da, HUC_df, combined_df, nc_df, HUC_gdf
ds

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 131 ms


<xarray.Dataset> Size: 403MB
Dimensions:                    (WBDHU12: 86617, time: 35)
Coordinates:
  * WBDHU12                    (WBDHU12) |S12 1MB b'010100020101' ... b'18100...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/24)
    total_gridded_area         (WBDHU12) float64 693kB ...
    Precip                     (WBDHU12, time) float32 12MB ...
    landmask                   (WBDHU12) float64 693kB ...
    deltaACCET                 (WBDHU12, time) float64 24MB ...
    deltaACSNOW                (WBDHU12, time) float64 24MB ...
    deltaSNEQV                 (WBDHU12, time) float32 12MB ...
    ...                         ...
    Area_sqkm                  (WBDHU12) float64 693kB ...
    totInflow                  (time, WBDHU12) float64 24MB ...
    bucket_depth               (time, WBDHU12) float64 24MB ...
    totqBucket                 (time, WBDHU12) float64 24MB ...
    totqSfcLatRunoff           (time, WBDHU12) float64 24MB ...
    totStreamflow              (time, WBDHU12) float64 24MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Rename variables

In [7]:
ds_out = ds.rename_vars(var_rename_dict)
ds_out

<xarray.Dataset> Size: 403MB
Dimensions:                    (WBDHU12: 86617, time: 35)
Coordinates:
  * WBDHU12                    (WBDHU12) |S12 1MB b'010100020101' ... b'18100...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/24)
    total_gridded_area         (WBDHU12) float64 693kB ...
    Precip                     (WBDHU12, time) float32 12MB ...
    LandFraction               (WBDHU12) float64 693kB ...
    ET                         (WBDHU12, time) float64 24MB ...
    Snowfall                   (WBDHU12, time) float64 24MB ...
    deltaSNEQV                 (WBDHU12, time) float32 12MB ...
    ...                         ...
    CatchmentArea              (WBDHU12) float64 693kB ...
    totInflow                  (time, WBDHU12) float64 24MB ...
    GWStore                    (time, WBDHU12) float64 24MB ...
    Baseflow                   (time, WBDHU12) float64 24MB ...
    Surfaceflow                (time, WBDHU12) float64 24MB ...
    totStreamflow              (time, WBDHU12) float64 24MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Rename Dimensions and coordinate variables

Using `xr.rename` instead of `xr.rename_dims` ensures that any coordinate variables are also renamed

In [8]:
ds_out = ds_out.rename(rename_dim_dict)
ds_out

<xarray.Dataset> Size: 403MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/24)
    total_gridded_area         (huc_id) float64 693kB ...
    Precip                     (huc_id, time) float32 12MB ...
    LandFraction               (huc_id) float64 693kB ...
    ET                         (huc_id, time) float64 24MB ...
    Snowfall                   (huc_id, time) float64 24MB ...
    deltaSNEQV                 (huc_id, time) float32 12MB ...
    ...                         ...
    CatchmentArea              (huc_id) float64 693kB ...
    totInflow                  (time, huc_id) float64 24MB ...
    GWStore                    (time, huc_id) float64 24MB ...
    Baseflow                   (time, huc_id) float64 24MB ...
    Surfaceflow                (time, huc_id) float64 24MB ...
    totStreamflow              (time, huc_id) float64 24MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Subset variables

In [9]:
ds_out = ds_out[list(var_rename_dict.values())]
ds_out

<xarray.Dataset> Size: 246MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/16)
    PrecipLand                 (huc_id, time) float32 12MB ...
    Snowfall                   (huc_id, time) float64 24MB ...
    Surfaceflow                (time, huc_id) float64 24MB ...
    Baseflow                   (time, huc_id) float64 24MB ...
    ET                         (huc_id, time) float64 24MB ...
    SWE                        (huc_id, time) float32 12MB ...
    ...                         ...
    CatchmentArea              (huc_id) float64 693kB ...
    Precip                     (huc_id, time) float32 12MB ...
    LandFraction               (huc_id) float64 693kB ...
    total_gridded_area         (huc_id) float64 693kB ...
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB ...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Change data types

In [10]:
for variable in ds_out.data_vars:
    if ds_out[variable].dtype == np.float64:
        print('Found a float64 for variable {0}'.format(variable))
        ds_out[variable] = ds_out[variable].astype(np.float32)
ds_out

Found a float64 for variable Snowfall
Found a float64 for variable Surfaceflow
Found a float64 for variable Baseflow
Found a float64 for variable ET
Found a float64 for variable SoilWater
Found a float64 for variable Recharge
Found a float64 for variable GWStore
Found a float64 for variable CatchmentArea
Found a float64 for variable LandFraction
Found a float64 for variable total_gridded_area


<xarray.Dataset> Size: 160MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/16)
    PrecipLand                 (huc_id, time) float32 12MB ...
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (time, huc_id) float32 12MB 0.3658 ... 6.764e-05
    Baseflow                   (time, huc_id) float32 12MB 30.68 ... 0.04509
    ET                         (huc_id, time) float32 12MB nan nan ... nan nan
    SWE                        (huc_id, time) float32 12MB ...
    ...                         ...
    CatchmentArea              (huc_id) float32 346kB 74.25 81.73 ... 941.5
    Precip                     (huc_id, time) float32 12MB ...
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB ...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB ...
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Re-order dimensions

In [11]:
for variable in ds_out.data_vars:
    #print(variable, ds_out[variable].dims)
    if ds_out[variable].dims == ('time', 'huc_id'):
        print('Var {0} not correct: {1}'.format(variable, ds_out[variable].dims))
        ds_out[variable] = ds_out[variable].transpose()
        #ds_out[variable] = ds_out[variable][['huc_id', 'time', variable]]
ds_out.load()
ds_out

Var Surfaceflow not correct: ('time', 'huc_id')
Var Baseflow not correct: ('time', 'huc_id')
Var GWStore not correct: ('time', 'huc_id')


<xarray.Dataset> Size: 160MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/16)
    PrecipLand                 (huc_id, time) float32 12MB nan nan ... nan nan
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (huc_id, time) float32 12MB 0.3658 ... 6.764e-05
    Baseflow                   (huc_id, time) float32 12MB 30.68 ... 0.04509
    ET                         (huc_id, time) float32 12MB nan nan ... nan nan
    SWE                        (huc_id, time) float32 12MB nan nan ... nan nan
    ...                         ...
    CatchmentArea              (huc_id) float32 346kB 74.25 81.73 ... 941.5
    Precip                     (huc_id, time) float32 12MB nan nan ... nan nan
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB nan nan ... nan nan
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB nan nan ... nan nan
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Landmask : 1=land, 0=water
    sr_x:         1
    sr_y:         1
    stagger:      M

### Set variable and global attributes

In [12]:
# Eliminate any unecessary variable attributes (such as spatial metadata)
for variable in ds_out.data_vars:
    if variable in var_atts_dict:
        ds_out[variable].attrs = var_atts_dict[variable]

# Now eliminate unnecessary global attributes 
ds_out.attrs = out_global_atts

ds_out

<xarray.Dataset> Size: 160MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/16)
    PrecipLand                 (huc_id, time) float32 12MB nan nan ... nan nan
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (huc_id, time) float32 12MB 0.3658 ... 6.764e-05
    Baseflow                   (huc_id, time) float32 12MB 30.68 ... 0.04509
    ET                         (huc_id, time) float32 12MB nan nan ... nan nan
    SWE                        (huc_id, time) float32 12MB nan nan ... nan nan
    ...                         ...
    CatchmentArea              (huc_id) float32 346kB 74.25 81.73 ... 941.5
    Precip                     (huc_id, time) float32 12MB nan nan ... nan nan
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB nan nan ... nan nan
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB nan nan ... nan nan
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### Reorganize time dimension to year and month dimensions

In [13]:
# Interpret times as strings - for later input to CSV files as a time index
datetime_strings = [pd.to_datetime(ds_out['time']).strftime('%Y%m%d%H')]

# year-month strings
yearmo_strings = [pd.to_datetime(ds_out['time']).strftime('%Y-%m')]
yearmo_strings

ds_out['yrmo'] = xr.DataArray(np.array(yearmo_strings, dtype='U'), dims=('yrmo_index', time_coord), name='yrmo')
ds_out

<xarray.Dataset> Size: 160MB
Dimensions:                    (huc_id: 86617, time: 35, yrmo_index: 1)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Dimensions without coordinates: yrmo_index
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 12MB nan nan ... nan nan
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (huc_id, time) float32 12MB 0.3658 ... 6.764e-05
    Baseflow                   (huc_id, time) float32 12MB 30.68 ... 0.04509
    ET                         (huc_id, time) float32 12MB nan nan ... nan nan
    SWE                        (huc_id, time) float32 12MB nan nan ... nan nan
    ...                         ...
    Precip                     (huc_id, time) float32 12MB nan nan ... nan nan
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB nan nan ... nan nan
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB nan nan ... nan nan
    yrmo                       (yrmo_index, time) <U7 980B '2011-01' ... '201...
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

In [14]:
# Set the nodata value
nodata_value = float(-9999)

In [15]:
#Drop unecessary dimension on `yrmo` variable
yrmo_dim_len = [len(ds_out[dim]) for dim in ds_out['yrmo'].dims]
if len(ds_out['yrmo'].dims) > 1 and 1 in yrmo_dim_len:
   remove_dims = [dim for dim in ds_out['yrmo'].dims if len(ds_out[dim])==1]
   print('Removing dimension(s) {0} from variable "yrmo".'.format(remove_dims))
   ds_out['yrmo'] = ds_out['yrmo'].squeeze()
ds_out

Removing dimension(s) ['yrmo_index'] from variable "yrmo".


<xarray.Dataset> Size: 160MB
Dimensions:                    (huc_id: 86617, time: 35)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 12MB nan nan ... nan nan
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (huc_id, time) float32 12MB 0.3658 ... 6.764e-05
    Baseflow                   (huc_id, time) float32 12MB 30.68 ... 0.04509
    ET                         (huc_id, time) float32 12MB nan nan ... nan nan
    SWE                        (huc_id, time) float32 12MB nan nan ... nan nan
    ...                         ...
    Precip                     (huc_id, time) float32 12MB nan nan ... nan nan
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB nan nan ... nan nan
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB nan nan ... nan nan
    yrmo                       (time) <U7 980B '2011-01' '2011-02' ... '2013-11'
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

5) Enforce a sort order on the variables in the dataset

In [17]:
# Build list of variables, sorted by 1D, then 2D alphabetical
sorted_varlist = ['yrmo', 'CatchmentArea', 'LandFraction', 'total_gridded_area']

# Build list of all 2D+ variables, sorted alphabetically no matter the case
sorted_varlist2 = [item for item in list(ds_out.data_vars) if item not in sorted_varlist]
sorted_varlist2.sort(key=str.casefold)

# Add the lists together
sorted_varlist += sorted_varlist2
assert len(list(ds_out.data_vars)) == len(sorted_varlist)
print('Found {0} variables. Sorted by number of dimensions and then alphabetically'.format(len(sorted_varlist)))
      
# Sort the variables in the dataset
out_ds = ds_out[sorted_varlist]
out_ds

Found 17 variables. Sorted by number of dimensions and then alphabetically


<xarray.Dataset> Size: 160MB
Dimensions:                    (time: 35, huc_id: 86617)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 280B 2011-01-01 ... 2013...
Data variables: (12/17)
    yrmo                       (time) <U7 980B '2011-01' '2011-02' ... '2013-11'
    CatchmentArea              (huc_id) float32 346kB 74.25 81.73 ... 941.5
    LandFraction               (huc_id) float32 346kB 0.9412 1.0 ... 1.0 0.06298
    total_gridded_area         (huc_id) float32 346kB 64.0 75.0 ... 72.0 58.0
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 12MB nan nan ... nan nan
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 12MB nan nan ... nan nan
    ...                         ...
    Recharge                   (huc_id, time) float32 12MB nan nan ... nan nan
    Snowfall                   (huc_id, time) float32 12MB nan nan ... nan nan
    SoilSat                    (huc_id, time) float32 12MB nan nan ... nan nan
    SoilWater                  (huc_id, time) float32 12MB nan nan ... nan nan
    Surfaceflow                (huc_id, time) float32 12MB 0.3658 ... 6.764e-05
    SWE                        (huc_id, time) float32 12MB nan nan ... nan nan
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 6) Fix NoData Issues

A decision was made to make all NaN values consistent between 10-year and 40-year Water Budget component files, using -9999.0 as the _FillValue in the netCDF files, and adding descirptions to variable attributes to identify what NaN means in each variable.

In [18]:
#### Fix variable attributes and encodings
output_encoding = {}
for data_var in out_ds.data_vars:
    print(data_var)

    # Fix nodata description attribute
    if data_var in ['PrecipLand', 
                    'Snowfall', 
                    'ET', 
                    'SWE', 
                    'SoilWater', 
                    'SoilSat', 
                    'Recharge', 
                    'Precip', 
                    'LandFraction', 
                    'total_gridded_area', 
                    'avgSOILM_wltadj_depthmean', 
                    'avgSOILSAT_wltadj_top1']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contains no land cells'
    elif data_var in ['CatchmentArea']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contains no WRF-Hydro catchment polygons'
    elif data_var in ['Surfaceflow', 
                      'Baseflow', 
                      'GWStore']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contain no WRF-Hydro flowlines.'
    else:
        continue
    print('\tnodata_description: {0}'.format(out_ds[data_var].attrs['nodata_description']))

    # Change NaN to nodata value
    nodata_mask = out_ds[data_var].isnull().data
    print('\tFound {0} nodata values'.format(nodata_mask.sum()))
    #display(out_ds[data_var].data[nodata_mask])
    out_ds[data_var].data[nodata_mask] = nodata_value
    #display(out_ds[data_var].data[nodata_mask])
    
    # Variable encoding
    out_ds[data_var].encoding['_FillValue'] = nodata_value
    output_encoding[data_var] = {'_FillValue':nodata_value}

    # Remove redundant missing value encoding
    if 'missing_value' in out_ds[data_var].encoding:
        del out_ds[data_var].encoding['missing_value']
    print('\t'.format(out_ds[data_var].encoding['_FillValue']))

yrmo
CatchmentArea
	nodata_description: HUC12 contains no WRF-Hydro catchment polygons
	Found 3343 nodata values
	
LandFraction
	nodata_description: HUC12 contains no land cells
	Found 13 nodata values
	
total_gridded_area
	nodata_description: HUC12 contains no land cells
	Found 13 nodata values
	
avgSOILM_wltadj_depthmean
	nodata_description: HUC12 contains no land cells
	Found 1992347 nodata values
	
avgSOILSAT_wltadj_top1
	nodata_description: HUC12 contains no land cells
	Found 1992347 nodata values
	
Baseflow
	nodata_description: HUC12 contain no WRF-Hydro flowlines.
	Found 118475 nodata values
	
ET
	nodata_description: HUC12 contains no land cells
	Found 1992347 nodata values
	
GWStore
	nodata_description: HUC12 contain no WRF-Hydro flowlines.
	Found 118475 nodata values
	
Precip
	nodata_description: HUC12 contains no land cells
	Found 1992347 nodata values
	
PrecipLand
	nodata_description: HUC12 contains no land cells
	Found 1992347 nodata values
	
Recharge
	nodata_description: H

In [19]:
# View the variable encodings
for var in out_ds.data_vars:
    print(var)
    for key,item in out_ds[var].encoding.items():
        print(f'    {key}: {item}')

yrmo
CatchmentArea
    _FillValue: -9999.0
LandFraction
    _FillValue: -9999.0
total_gridded_area
    _FillValue: -9999.0
avgSOILM_wltadj_depthmean
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /caldera/hovenweep/projects/usgs/water/impd/hytest/working/niwaa_wrfhydro_monthly_huc12_aggregations/merge_out/CONUS_HUC12_WB_combined_19791001_20220930.nc
    original_shape: (86617, 35)
    _FillValue: -9999.0
avgSOILSAT_wltadj_top1
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /caldera/hovenweep/projects/usgs/water/impd/hytest/working/niwaa_wrfhydro_monthly_huc12_aggregations/merge_out/CONUS_HUC12_WB_combined_19791001_20220930.nc
    original_shape: (86617, 35)
  

In [20]:
%%time

print('  Writing output to {0}'.format(out_nc))
out_ds.compute()
out_ds.to_netcdf(out_nc, encoding=output_encoding)
out_ds.close()

  Writing output to /caldera/hovenweep/projects/usgs/water/impd/hytest/working/niwaa_wrfhydro_monthly_huc12_aggregations/final_out/fresh/huc12_monthly_wb_iwaa_wrfhydro_WY2011_2013.nc
CPU times: user 341 ms, sys: 226 ms, total: 567 ms
Wall time: 689 ms


### Clean up

In [21]:
ds.close()
ds_out.close()
print('Process completed in {0: 3.2f} seconds.'.format(time.time()-tic))

Process completed in  232.84 seconds.
